# Experiment 8 - Colloborative Filtering 


In [7]:
# Importing required libraries

import pandas as pd
import numpy as np
import math
import numpy as np
from sklearn.metrics import pairwise_distances

In [8]:
# Loading data set to see the structure of the dataset



df = pd.read_csv('/ml-100k/u.data',sep = "\t",header= None)

print(df.head())

     0    1  2          3
0  196  242  3  881250949
1  186  302  3  891717742
2   22  377  1  878887116
3  244   51  2  880606923
4  166  346  1  886397596


In [9]:
# Calculating item_id rating prediction based on user-based collobrative filtering

def get_rating(u_data,u,i,no_neigh):
    
    user_id = u
    item_id = i
    df = u_data

    uniq_users = len(set(df.iloc[:,0]))
    uniq_movies = len(set(df.iloc[:,1]))
    
    print("Unique users :",uniq_users)
    print("Unique movies :",uniq_movies)

    # Creating Data matrix from values acquired

    data_mat = np.full((uniq_users,uniq_movies),0)

    for index,value in df.iterrows():

        data_mat[value[0]-1,value[1]-1] = value[2]


    us_sim_matrix = 1-pairwise_distances(data_mat, metric="cosine")
    it_sim_matrix = 1-pairwise_distances(np.transpose(data_mat),metric = "cosine")
    

    #  Getting the k no of most similar neighbours
    no_of_neigh = no_neigh
    us_nrst_neighs = []
    it_nrst_neighs = []

    us_neighs = [(i) for i in range(uniq_users)]
    it_neighs = [(i) for i in range(uniq_movies)]

    us_nrst_neighs = sorted(zip(us_sim_matrix[user_id-1],us_neighs),reverse = True)[1:(no_of_neigh+1)]

    it_nrst_neighs = sorted(zip(it_sim_matrix[item_id-1],it_neighs),reverse = True)[1:(no_of_neigh+1)]
    
    # Calculating user based rating for item
    us_mean_rating = np.mean(data_mat[user_id-1,:])

    us_sum_denom = float(0)
    us_sum_numer = float(0)
    for sim in us_nrst_neighs:
        
        us_sum_denom += sim[0]
        us_mean_user_rating  = np.mean(data_mat[sim[1],:])
        us_sum_numer += (sim[0]*(data_mat[sim[1],item_id-1] - us_mean_user_rating))

    us_rating = (us_mean_rating + (us_sum_numer/us_sum_denom))

    
    if us_rating < 0:
        us_rating = 0
    
    elif us_rating > 5:
        us_rating =  5 
    
    print("User based rating for user " + str(user_id) + " rating for item "+ str(item_id) +" is :" +str(us_rating))

    # Calculating item based rating for item
     
    it_mean_rating = np.mean(data_mat[:,item_id-1])
    it_sum_denom = float(0)
    it_sum_numer = float(0)

    for sim in it_nrst_neighs:

        it_sum_denom += sim[0]

        it_mean_rating = np.mean(data_mat[:,sim[1]])

        us_sum_numer += (sim[0] *(data_mat[user_id-1,sim[1]] - it_mean_rating))

    it_rating = (it_mean_rating + (it_sum_numer/it_sum_denom))

    if it_rating < 0:
        it_rating = 0
    
    elif it_rating > 5:
        it_rating =  5 

    print("Item based rating for user " + str(user_id) + " rating for item "+ str(item_id) +" is :" +str(it_rating))



    





In [10]:
get_rating(df,250,100,10)

Unique users : 943
Unique movies : 1682
User based rating for user 250 rating for item 100 is :4.302035747340593
Item based rating for user 250 rating for item 100 is :1.2354188759278897
